In [ ]:
# need to install shapely

import pandas as pd
import geopandas as gpd
import numpy as np
import os
import json
from shapely.geometry import Point


In [ ]:
os.chdir('/mnt/c/Users/User/notebook/jupyterenv/generic-real-estate-consulting-project-18/')


In [ ]:
sf_SA2 = pd.read_csv("data/curated/shapefiles/SA2_shape_clean.csv")
rental_data = pd.read_csv("data/curated/rental/rental_properties_cleaned.csv")

In [ ]:
sf_SA2['geometry'] = gpd.GeoSeries.from_wkt(sf_SA2['geometry'])

gdf_SA2 = gpd.GeoDataFrame(sf_SA2, geometry= 'geometry')

In [ ]:
gdf_SA2

In [ ]:
rental_data

In [ ]:
rental_data['property_type'].unique()

In [ ]:
rental_data_house_apt = rental_data[rental_data['property_type'] != 'storage unit']
rental_data_house_apt = rental_data_house_apt[rental_data_house_apt['property_type'] != 'Carspace']

In [ ]:
property_price = rental_data_house_apt[['name', 'cost_text', 'property_type', 'coordinates']]

property_price = property_price.reset_index()


property_price.drop(columns=property_price.columns[0], axis=1, inplace=True)


In [ ]:
new_coordinates = []
for i in property_price['coordinates']:
    res = i.strip('][').split(', ')
    new_coordinates.append(res)

    
result = []
 
for sublist in new_coordinates:
    float_sublist = []
    for x in sublist:
        float_sublist.append(float(x))
    result.append(float_sublist)

print(result)

In [ ]:
new_coordinates = pd.DataFrame(result, columns = ['Longitude', 'Latitude'])


property_price['Longitude'] = new_coordinates['Longitude']
property_price['Latitude'] = new_coordinates['Latitude']

property_price.drop(columns=property_price.columns[0], axis=1, inplace=True)

In [ ]:
property_price['New_Coordinates'] = list(zip(property_price.Latitude, property_price.Longitude))

property_price.drop(columns=property_price.columns[2], axis=1, inplace=True)

In [ ]:
property_price['New_Coordinates'] = property_price['New_Coordinates'].apply(Point)

property_price

In [ ]:
gdf_prop_price = gpd.GeoDataFrame(property_price, geometry='New_Coordinates')

gdf_prop_price

In [ ]:
gdf_prop_price.drop(['Longitude', 'Latitude'], axis = 1)


In [ ]:
gdf_SA2

In [ ]:
pointInPolys = gpd.tools.sjoin(gdf_prop_price, gdf_SA2, predicate="within", how='left')

In [ ]:
pointInPolys